In [54]:
#Tool creation
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

In [55]:
#Inbuilt wikipedia tools
api_wrapper_wiki = WikipediaAPIWrapper (top_k_results=1,
                                        doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper= api_wrapper_wiki)
wiki.name


'wikipedia'

In [56]:
#Inbuilt Arxiv tools
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1,
                                        max_document_length=200)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv.name

'arxiv'

In [57]:
tools = [wiki,arxiv]

In [58]:
#Custom RAG Tool Creation

from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

loader =WebBaseLoader("https://docs.smith.langchain.com/")
docs= loader.load()
document = RecursiveCharacterTextSplitter(chunk_size = 1000,
                                          chunk_overlap=200).split_documents(docs)
document

[Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en'}, page_content='Get started with LangSmith | 🦜️🛠️ LangSmith'),
 Document(metadata={'source': 'https://docs.smith.langchain.com/', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en'}, page_content='Skip to main contentWe are growing and hiring for multiple roles for LangChain, LangGraph and LangSmith.  Join our team!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceChangelogCloud architec

In [59]:
vectorstore = FAISS.from_documents(document,OpenAIEmbeddings())
vectorstore

In [60]:
retriever = vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x757f8a742ef0>, search_kwargs={})

In [86]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever,"Langsmith-Search",
                                        "Use this tool to search about LangSmith's features, architecture, or usage.")
retriever_tool.name

'Langsmith-Search'

In [87]:
tools = [arxiv,retriever_tool,wiki]
tools

[ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=4000)),
 Tool(name='Langsmith-Search', description="Use this tool to search about LangSmith's features, architecture, or usage.", args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x757fd9d30f70>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x757f8a742ef0>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial

In [88]:
#Agent Executor

from langchain_openai import ChatOpenAI 
from dotenv import load_dotenv
import openai
load_dotenv()
import os
openai_api_key = os.getenv("OPENAI_API_KEY")
llm = ChatOpenAI(api_key=openai_api_key,model='gpt-4.1-2025-04-14')
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x757f8a490d90>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x757f8a339780>, root_client=<openai.OpenAI object at 0x757f8a490490>, root_async_client=<openai.AsyncOpenAI object at 0x757f8a490a90>, model_name='gpt-4.1-2025-04-14', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [89]:
#Prompt temmplate
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [90]:
#Agents

from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [91]:
#Agent Executor
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(
    agent = agent,
    tools = tools,
    verbose = True
)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [93]:
agent_executor.invoke({"input": "Search for LangSmith features."})



> Entering new AgentExecutor chain...

Invoking: `Langsmith-Search` with `{'query': 'LangSmith features'}`


LangSmith is a platform for building production-grade LLM applications.
It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.
ObservabilityAnalyze traces in LangSmith and configure metrics, dashboards, alerts based on these.EvalsEvaluate your application over production traffic — score application performance and get human feedback on your data.Prompt EngineeringIterate on prompts, with automatic version control and collaboration features.
LangSmith + LangChain OSSLangSmith is framework-agnostic — it can be used with or without LangChain's open source frameworks langchain and langgraph.If you are using either of these, you can enable LangSmith tracing with a single environment variable.
For more see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
Observability​

Get started 

{'input': 'Search for LangSmith features.',
 'output': "Here are the key features of LangSmith:\n\n1. Observability:\n   - Analyze traces in LangSmith and set up metrics, dashboards, and alerts.\n   - LLM-native observability lets you debug and gain insights at any stage of development (from prototyping to production).\n\n2. Evaluation:\n   - Evaluate your application's performance over production traffic.\n   - Score application output and gather human feedback.\n\n3. Prompt Engineering:\n   - Iterate on and version prompts with built-in version control and collaboration tools.\n\n4. Integration with LangChain OSS:\n   - Framework-agnostic, but integrates easily with LangChain and LangGraph for seamless tracing.\n\n5. Deployment & Administration:\n   - Features for deployment (especially with LangGraph platform) and self-hosting.\n   - Administration tools for access control and authentication.\n\n6. Collaboration:\n   - Built-in features for teams to collaborate on prompt engineering

In [70]:
agent_executor.invoke({"input":"Tell me about chelsea fc?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Chelsea F.C.'}`


Page: Chelsea F.C.
Summary: Chelsea Football Club is a professional football club based in Fulham, West London, England. The club was founded in 1905 and named after neighbouring area Chelsea. They coChelsea Football Club is a professional football club based in Fulham, West London, England. Founded in 1905, Chelsea is one of England's most successful and widely supported football teams.

Key points about Chelsea FC:

- Founded: 1905
- Location: Fulham, West London
- Stadium: Stamford Bridge
- Achievements: Chelsea has won multiple Premier League titles, FA Cups, and has also achieved success in European competitions, including winning the UEFA Champions League.
- Nickname: "The Blues"
- Notable for its passionate fan base and history of both domestic and European success.

If you want more specific information such as recent achievements, current players, or club history in detail, let me

{'input': 'Tell me about chelsea fc?',
 'output': 'Chelsea Football Club is a professional football club based in Fulham, West London, England. Founded in 1905, Chelsea is one of England\'s most successful and widely supported football teams.\n\nKey points about Chelsea FC:\n\n- Founded: 1905\n- Location: Fulham, West London\n- Stadium: Stamford Bridge\n- Achievements: Chelsea has won multiple Premier League titles, FA Cups, and has also achieved success in European competitions, including winning the UEFA Champions League.\n- Nickname: "The Blues"\n- Notable for its passionate fan base and history of both domestic and European success.\n\nIf you want more specific information such as recent achievements, current players, or club history in detail, let me know!'}

In [ ]:
agent_executor.invoke({"input":"What's the paper 1706.03762 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks in an encoder-decoder configuration. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer, based
solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experiments on two machine translation tasks show these models to be
superior in quality while being more parallelizable and requiring significantly
less time to train. Our model achieves 28.4 BLEU on the WMT 2014
English-to-German translation task, improving over the existing best results,
including ensembles by over 

{'input': "What's the paper 1706.03762 about?",
 'output': 'The paper "Attention Is All You Need" (arXiv:1706.03762) introduces the Transformer model—an innovative neural network architecture for sequence transduction tasks, such as machine translation. Unlike previous models that relied on recurrent or convolutional layers, the Transformer is based solely on attention mechanisms, discarding recurrence and convolutions entirely.\n\nKey points from the paper:\n- The Transformer architecture achieves superior quality on machine translation tasks (e.g., English-to-German and English-to-French) while being highly parallelizable and significantly faster to train than previous models.\n- It set new state-of-the-art BLEU scores for major translation benchmarks using fewer training resources.\n- The model also generalizes well to other tasks, like English constituency parsing.\n\nThe Transformer has since become foundational in modern deep learning, especially for natural language processing, 